In [129]:
import nltk
>>> nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [70]:
# extract the resulting vocabulary
terms = vectorizer.get_feature_names()
print("Vocabulary has %d distinct terms" % len(terms))

Vocabulary has 2234 distinct terms


In [73]:
import operator
def rank_terms( A, terms ):
    # get the sums over each column
    sums = A.sum(axis=0)
    # map weights to the terms
    weights = {}
    for col, term in enumerate(terms):
        weights[term] = sums[0,col]
    # rank the terms by their weight over all documents
    return sorted(weights.items(), key=operator.itemgetter(1), reverse=True)

In [68]:
#display the rankings
ranking = rank_terms( A, terms )
for i, pair in enumerate( ranking[0:20] ):
    print( "%02d. %s (%.2f)" % ( i+1, pair[0], pair[1] ) )

01. and (38.32)
02. in (24.05)
03. data (22.34)
04. of (16.18)
05. to (15.45)
06. learning (15.12)
07. the (14.98)
08. with (12.82)
09. machine (12.00)
10. my (10.76)
11. experience (10.25)
12. analysis (9.63)
13. google (9.02)
14. statistical (8.73)
15. for (8.67)
16. science (8.34)
17. on (8.33)
18. at (8.26)
19. management (8.01)
20. modeling (7.70)


## Exploring Linkedin Summaries using Topic Modeling


In [2]:
import pandas as pd
import sklearn
import numpy as np
import matplotlib as matplot
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', -1)

In [137]:
#Load the Google Data Scientist Linkedin Data
googleDatasci = pd.read_csv("Datasets/googledatasci.csv")

#Remove the first column
del googleDatasci['Unnamed: 0']

#Select only the rows with Google as company
googleDatasci = googleDatasci.loc[(googleDatasci.company == 'Google')]

#Remove any row that doesn't contain data scientist in the headline
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Technical Program Manager") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Supply chain & Operations leader") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("DevIntel on Developer Products") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("China Display Product Lead") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Sales Operations Senior") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("APM Intern") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Trust and Safety manager") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("User Growth, Google Play") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Ads Solutions Consultant") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Regional Customer Engineer") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Software Engineering Intern") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Product Manager") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Software Developer") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Software Engineer") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Software engineer") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Opportunity Seeker") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Student at University of Toronto") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Google Policy Fellow") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Science Advocate") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Engineer") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Account Manager Display") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Customer Engineering Manager") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Strategist") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Program Manager") == False]

#print out the dataset
#googleDatasci

In [143]:
#Select the summaries column to write a text file to build the initial corpus
googleDatasummary = googleDatasci['summary']

#Remove any unnecessary characters 
googleDatasci['summary'] = googleDatasci['summary'].replace(r'\t', '', regex=True) 
googleDatasci['summary'] = googleDatasci['summary'].replace(r'\n', ' ', regex=True) 
googleDatasci['summary'] = googleDatasci['summary'].replace(r'\...', ' ', regex=True)
googleDatasci['summary'] = googleDatasci['summary'].replace(r'\..', ' ', regex=True)
googleDatasci['summary'] = googleDatasci['summary'].replace('●', '', regex=True) 
googleDatasci['summary'] = googleDatasci['summary'].replace('•', '', regex=True)
googleDatasci['summary'] = googleDatasci['summary'].replace('-', '', regex=True) 
googleDatasci['summary'] = googleDatasci['summary'].replace('&', '', regex=True)
googleDatasci['summary'] = googleDatasci['summary'].replace('$', '', regex=True)
googleDatasci['summary'] = googleDatasci['summary'].replace(',', '', regex=True)


#googleDatasci = googleDatasci.translate(None, "(){}<>")

#
googleDatasci['summary'] = googleDatasci['summary'].str.lower()

#Remove any row with a NaN
googleDatasci['summary'] = googleDatasci['summary'].dropna()

googleDatasci

name  \
1    Aaron Roan                       
2    Abhilasha S.                     
3    Abhilash Mittapalli              
5    Adam Kalman, Ph.D.               
6    Aditya Padala                    
7    Adrián González Álvarez          
9    Adrish Sannyasi                  
10   Ahmad Emad                       
12   Aki Matsushima                   
18   Alex Chinien                     
25   Amaan Mehrabian                  
27   Andres Lindner                   
28   Angshuman Saha                   
30   Anna Katrine Jørgensen           
34   Arik Motskin                     
35   Arthur Nen                       
36   Ashish Tendulkar                 
39   Alexander Yakubovich             
41   Brian Bahmanyar                  
45   Carlos Riquelme                  
46   Cathy Yuen Yi Lee                
48   Christopher Erickson             
51   Chao Ding                        
56   Christian Howard                 
59   Christopher Stewart, Ph.D.       
61   Kevin Corti                      
62   Crystal Jenkins                  
66   David Remus                      
69   Debidatta Dwibedi                
70   Richard Demsyn-Jones             
..                    ...             
311  Kaushik Subramanian              
313  Jan Vlachy                       
314  Wan Chen                         
319  William Lane                     
320  Wynn Vonnegut                    
321  Xia Li                           
322  Xianzhi (Helen) Wang             
323  Xiaofan Xu                       
324  Xiaojing Huang                   
325  Xiaojun Di                       
326  Xiaoli Yan                       
327  Xiaoxiao Wang                    
328  Eric Li                          
329  Xiaoyu Dai                       
330  Eric Bai                         
332  Jackie (Rongjing) Xiang          
334  Yading Song, PhD                 
338  Yaohui Zeng                      
341  Yi Deng                          
342  Yipeng Yang                      
343  Yiren Lu                         
344  Yoann ASSAYAG                    
345  Yuan Jin                         
346  Yuao Hu                          
347  Yue Shi                          
349  Yunzhu (Anny) Chen               
353  Zhenyu Lu                        
354  Zhifan (Vivienne) Zhang, Ph.D.   
355  Zhuo Z.                          
356  Zynab Ali                        

                                                                                                                                                                                                                                                                certifications  \
1    Preventing Workplace Harassment                                                                                                                                                                                                                                             
2    Python, SQL; Green Belt                                                                                                                                                                                                                                                     
3    Programming in HTML5 with JavaScript and CSS3; Querying SQL server                                                                                                                                                                                                          
5    Machine Learning; The Data Scientist's Toolbox; Mathematics Teacher, grades 7-12; AP Statistics Teacher                                                                                                                                                                     
6    Kaggle R Tutorial on Machine Learning                                                                                                                                                                                                        

In [135]:
from nltk.tokenize import WhitespaceTokenizer
ws_tokenizer = WhitespaceTokenizer()

article = googleDatasci['summary'][1]

# tokenize example document
nyt_ws_tokens = ws_tokenizer.tokenize(article.lower())

#print (nyt_ws_tokens)

## Term Weighting using TF-IDF

In [145]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import WhitespaceTokenizer

def preprocess_article_content(text_df):
    """
    Simple preprocessing pipeline which uses RegExp, sets basic token requirements, and removes stop words.
    """
    #print 'preprocessing article text...'

    # tokenizer, stops, and stemmer
    tokenizer = RegexpTokenizer(r'\w+')
    stop_words = set(stopwords.words('english'))  # can add more stop words to this set
    stemmer = SnowballStemmer('english')

    # process articles
    article_list = []
    for row, article in enumerate(text_df['summary']):
        cleaned_tokens = []
        tokens = tokenizer.tokenize(str(article))
        for token in tokens:
            if token not in stop_words:
                if len(token) > 0 and len(token) < 20: # removes non words
                    if not token[0].isdigit() and not token[-1].isdigit(): # removes numbers
                        stemmed_tokens = stemmer.stem(token)
                        cleaned_tokens.append(stemmed_tokens)
        # add process article
        article_list.append(' '.join(wd for wd in cleaned_tokens))

    # echo results and return
    #print 'preprocessed content for %d articles' % len(article_list)
    return article_list

# process articles
processed_article_list = preprocess_article_content(googleDatasci)

# vectorize the articles and compute count matrix
from sklearn.feature_extraction.text import TfidfVectorizer
tf_vectorizer = TfidfVectorizer()
tfidf_article_matrix = tf_vectorizer.fit_transform(processed_article_list)

tfidf_article_matrix.shape

(193, 1807)